In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [3]:
max_size = 57
max_length = 56

In [4]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [5]:
import mthree
from qiskit.test.mock import FakeManhattan

In [6]:
xz_nation_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    mit = mthree.M3Mitigation(FakeManhattan())
    mit.single_qubit_cals = meas_fitter.cal_matrices
    xz_nation_mitigator_list.append(mit)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_nation_mitigator_list: ", len(xz_nation_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of xz_nation_mitigator_list:  56


In [7]:
zx_nation_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    mit = mthree.M3Mitigation(FakeManhattan())
    mit.single_qubit_cals = meas_fitter.cal_matrices
    zx_nation_mitigator_list.append(mit)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_nation_mitigator_list: ", len(zx_nation_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of zx_nation_mitigator_list:  56


In [8]:
nation_hist_tuple_list = []
xz_nation_total_times = []
zx_nation_total_times = []
for i in range(29):
    print(i + 2, "qubits")
    meas_layout = [i for i in range(i + 2)]
    
    t1 = time.perf_counter()
    nation_hist_tuple_list.append( xz_nation_mitigator_list[i].apply_correction(raw_hist_list[2 * i], 
                                                                       meas_layout, 
                                                                       distance=None,
                                                                       method='iterative',
                                                                       max_iter=25, 
                                                                       tol=1e-5,
                                                                       return_mitigation_overhead=True,
                                                                       details=True) )
    t2 = time.perf_counter()
    xz_nation_total_times.append(t2 - t1)
    print("xz", t2 - t1, "s")
    
    t3 = time.perf_counter()
    nation_hist_tuple_list.append( zx_nation_mitigator_list[i].apply_correction(raw_hist_list[2 * i + 1], 
                                                                       meas_layout, 
                                                                       distance=None,
                                                                       method='iterative',
                                                                       max_iter=25, 
                                                                       tol=1e-5,
                                                                       return_mitigation_overhead=True,
                                                                       details=True) )
    t4 = time.perf_counter()
    zx_nation_total_times.append(t4 - t3)
    print("zx", t4 - t3, "s")
    
    print()

2 qubits
xz 0.002942315000000306 s
zx 0.0025565000000007387 s

3 qubits
xz 0.003596662000001416 s
zx 0.0023117560000009973 s

4 qubits
xz 0.00620889899999888 s
zx 0.005476338999999442 s

5 qubits
xz 0.0037291950000000185 s
zx 0.003384313999999833 s

6 qubits
xz 0.006252922000001604 s
zx 0.005341793999999567 s

7 qubits
xz 0.0074317999999991 s
zx 0.015056847000000317 s

8 qubits
xz 0.023282721999999367 s
zx 0.037620580999998765 s

9 qubits
xz 0.051457943999999145 s
zx 0.10793775800000027 s

10 qubits
xz 0.14147281000000156 s
zx 0.1944828480000016 s

11 qubits
xz 0.1980832380000006 s
zx 0.5488970630000001 s

12 qubits
xz 0.5353693309999983 s
zx 1.1217726900000002 s

13 qubits
xz 0.9925784159999971 s
zx 1.42001651 s

14 qubits
xz 2.5187476019999977 s
zx 3.7429123779999998 s

15 qubits
xz 3.7182513640000003 s
zx 5.735519133999997 s

16 qubits
xz 5.410762527999999 s
zx 7.003744378 s

17 qubits
xz 8.140039127999998 s
zx 11.262174778000002 s

18 qubits
xz 12.166726834999992 s
zx 17.445713282 

In [9]:
nation_hist_list = []
nation_info_list = []
for i in range(29 * 2):
    nation_hist_list.append(nation_hist_tuple_list[i][0])
    nation_info_list.append(nation_hist_tuple_list[i][1])

In [10]:
with open("./pkls/nation_hist_list.pkl", "wb") as f:
    pickle.dump(nation_hist_list, f)

In [11]:
with open("./pkls/nation_info_list.pkl", "wb") as f:
    pickle.dump(nation_info_list, f)

In [12]:
with open("./pkls/nation_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_nation_total_times, "zx": zx_nation_total_times}, f)